In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from paths import Node
from paths import Path
from paths import pathsIntersect
import time 

In [ ]:
#physical constants
L = 15 #length of sensor array 
W = 20 #width of the car 
V = 2 #speed 
B = 1 #baud rate 
VB = V/B #speed distance/cycle
maxE = 0.5*L

#load the track
df = pd.read_csv("track.csv")
TX = np.array(df["X"].to_list())
TY = np.array(df["Y"].to_list())

#track constants
target = [261, -12] #destination position
pos = [0,-12] #car starting position
heading = np.pi/2 #initial heading
startDist = 261 #initial distance from destination

#runtime constants
maxTime = 2 #maximum runtime seconds
maxDist = 3 #maximum distance from destination

#PID constants
KP = (np.pi)*(W/2)/(0.5*L)
KI = 0
KD = 0

In [ ]:
#heading of car
def theta(vdiff, heading):
    heading += (2/W)*vdiff
    return heading

#velocity vector of car
def v(heading):
    return VB*np.cos(heading),  VB*np.sin(heading)

#coordinates of car
def p(heading, pos):
    x, y = v(heading)
    pos[0] = pos[0] + x
    pos[1] = pos[1] + y
    return pos[0], pos[1]


In [ ]:
#rotate point 
def rotate(px, py, cx, cy, heading):
    ox = cx
    oy = cy
    qx = ox + np.cos(heading) * (px - ox) - np.sin(heading) * (py - oy)
    qy = oy + np.sin(heading) * (px - ox) + np.cos(heading) * (py - oy)
    return qx, qy

#error function
def err(pos, heading):

    maxE = 0.5*L
    phi =  heading - 0.5*np.pi
    cx = pos[0]
    cy = pos[1]
    ax = cx + 0.5*L*np.cos(phi)
    ay = cx + 0.5*L*np.sin(phi)
    bx = cx - 0.5*L*np.cos(phi)
    by = cx - 0.5*L*np.sin(phi)

    #select points of interest
    p1x = float('inf')
    p1y = float('inf')
    p2x = float('inf')
    p2y = float('inf')

    #rotate the entire track and crop out parts that are far from the car
    rx = np.array([])
    ry = np.array([])
    for i in range(len(TX)):
        dist = ( (cx - TX[i])**2 + (cy - TY[i])**2 )**0.5
        if (dist < 0.5*L):
            qx, qy = rotate(TX[i], TY[i], cx, cy, -1*phi)
            rx = np.append(rx, qx)
            ry = np.append(ry, qy)
    
    #if no points close to car, return max error
    if len(rx) == 0 or len(ry) == 0:
        return maxE
    
    #find the point of the track that the sensor intersects

    temp = ry
    i1 = np.argmin(abs(cy - temp))
    temp[i1] = float('inf')
    i2 = np.argmin(abs(cy - temp))

    p1x = rx[i1]
    p1y = ry[i1]
    p2x = rx[i2]
    p2y = ry[i2]

    nodeA = Node(ax, ay)
    nodeB = Node(bx, by)
    node1 = Node(p1x, p1y)
    node2 = Node(p2x, p2y)
    pathSensor = Path(nodeA, nodeB)
    pathTrack = Path(node1, node2)

    intersect, ix, iy = pathsIntersect(pathSensor, pathTrack)
    if not intersect: return maxE

    #calculate and return the error
    return cx - ix

In [ ]:
s = time.time() #record runtime

dist = startDist #initialize distance from destination
diff = 0 #initial wheel speed differebce
prev_e = 0 #initialize derivative
integral = 0 #initialize integral

pos_x = np.array([])
pos_y = np.array([])
errs = np.array([])

while(dist > maxDist and time.time() - s < maxTime):
    #append the position history
    pos_x = np.append(pos_x, pos[0])
    pos_y = np.append(pos_y, pos[1])

    #find the car position
    heading = theta(diff, heading)
    pos[0], pos[1] = p(heading, pos)
   
    #find error
    e = err(pos, heading)
    errs = np.append(errs, e)

    #find wheel speed difference
    prop = KP*e
    int = KI*integral
    der = KD*(e - prev_e)
    diff = prop + int + der
    
    #update pid
    integral += e
    prev_e = e

    #check distance from destination
    dist = ((target[0] - pos[0])**2 + (target[1] - pos[1])**2)**0.5

plt.figure()
plt.title("PID Output")
plt.plot(TX, TY)  
plt.plot(pos_x[:], pos_y[:])

plt.figure()
plt.title("PID Error")
plt.scatter(np.linspace(0, len(errs), len(errs)), errs)

